LLM 实际上并不是重复预测下一个单词，而是重复预测下一个 token 。对于一个句子，语言模型会先使用分词器将其拆分为一个个 token ，而不是原始的单词。对于生僻词，可能会拆分为多个 token 。
这样可以大幅降低字典规模，提高模型训练和推断的效率。例如，对于 "Learning new things is fun!" 这句话，每个单词都被转换为一个 token ，而对于较少使用的单词，如 "Prompting as powerful developer tool"，单词 "prompting" 会被拆分为三个 token，即"prom"、"pt"和"ing"。

# 三种不同粒度的分词方法
+ word-level
定义：以单词为单位进行分词。
优点：直观，易于理解。
缺点：词表大，难以处理生僻词和新词（OOV 问题）。
例子：
"Learning new things is fun!" → ["Learning", "new", "things", "is", "fun", "!"]
+ character-level
定义：以单个字符为单位进行分词。
优点：词表小，无 OOV 问题。
缺点：序列变长，语义信息弱。
例子：
"fun" → ["f", "u", "n"]
+ Subword-level（子词级分词）
定义：将单词拆分为更小的子词单元（如 BPE、WordPiece、Unigram）。
优点：兼顾词表大小和 OOV 问题，能处理新词。
缺点：实现复杂。
例子：
"prompting" → ["prom", "pt", "ing"]

1. BPE（Byte Pair Encoding）
原理：BPE 最初用于数据压缩，后被用于 NLP 分词。它通过不断合并文本中出现频率最高的字符对，逐步构建子词单元。
步骤：
+ 将文本拆成单字符序列。
+ 统计所有相邻字符对的出现频率。
+ 合并出现频率最高的字符对，形成新的子词。
+ 重复步骤 2-3，直到达到预设的词表大小或没有高频对可合并。
优点：能有效减少 OOV 问题，词表大小可控。
应用：GPT、RoBERTa 等模型。
2. WordPiece
原理：WordPiece 由 Google 提出，最早用于机器翻译，后成为 BERT 的分词方法。与 BPE 类似，但合并策略基于最大化训练数据的似然概率。
步骤：
+ 初始化词表为所有字符。
+ 迭代地选择能最大提升训练数据似然的子词合并。
+ 直到词表达到目标大小。
优点：能更好地平衡词表大小与覆盖率，适合大规模语料。
应用：BERT、ALBERT、DistilBERT 等。
3. Unigram Language Model
原理：Unigram LM 由 Google 提出，常用于 SentencePiece。它假设每个子词的出现是独立的，通过概率建模选择最优子词集合。
步骤：
+ 初始化一个大词表（所有可能的子词）。
+ 通过 EM 算法估算每个子词的概率。
+ 迭代地删除概率最低的子词，直到词表缩小到目标大小。
+ 对新词分词时，选择概率乘积最大的子词序列。
优点：分词灵活，能自动适应不同语言和文本。
应用：SentencePiece（Google）、T5、XLNet 等。

面试题与参考答案
题目1：请简述 BPE、WordPiece 和 Unigram LM 的主要区别。
参考答案：

BPE 通过合并高频字符对构建子词，合并策略基于频率。
WordPiece 也合并子词，但合并策略基于最大化训练数据的似然概率。
Unigram LM 假设子词独立出现，通过概率建模和 EM 算法筛选子词，最终选择概率最大的分词方式。
题目2：为什么现代 NLP 模型更倾向于使用子词级分词方法？
参考答案：
子词级分词方法（如 BPE、WordPiece、Unigram LM）能有效平衡词表大小和 OOV 问题。它们既能处理常见词汇，也能将生僻词拆分为已知子词，提升模型泛化能力和效率。

题目3：请简述 BPE 分词的基本流程，并说明其优缺点。
参考答案：
流程：

文本拆为单字符序列。
统计所有相邻字符对频率。
合并频率最高的字符对，形成新子词。
重复直到词表大小满足要求。
优点：词表可控，能处理新词。
缺点：合并策略简单，未考虑上下文或概率信息。

In [ ]:
# 安装：pip install tokenizers
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# 初始化空的 BPE 模型
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 训练 BPE
trainer = trainers.BpeTrainer(vocab_size=1000, min_frequency=2)
files = ["your_text.txt"]  # 训练语料路径
tokenizer.train(files, trainer)

# 分词
output = tokenizer.encode("Learning new things is fun!")
print(output.tokens)

In [ ]:
# 初始化空的 WordPiece 模型
tokenizer = Tokenizer(models.WordPiece())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 训练 WordPiece
trainer = trainers.WordPieceTrainer(vocab_size=1000, min_frequency=2)
files = ["your_text.txt"]
tokenizer.train(files, trainer)

# 分词
output = tokenizer.encode("Learning new things is fun!")
print(output.tokens)

In [ ]:
# 安装：pip install sentencepiece
import sentencepiece as spm

# 训练 Unigram 模型
spm.SentencePieceTrainer.Train('--input=your_text.txt --model_prefix=unigram --vocab_size=1000 --model_type=unigram')

# 加载模型
sp = spm.SentencePieceProcessor()
sp.load('unigram.model')

# 分词
print(sp.encode('Learning new things is fun!', out_type=str))